# Agenda

We have three goals for today:

- Build our first simulation
- Extract a probability distribution from that simulation
- Learn good coding practices: debugging and docstrings

# The Lady Tasting Tea

The English are very serious about their tea. In the mid-1920s, Ronald Fisher began work on what is now a famous problem in the history of statistics. Muriel Bristol was an algae scientist and an associate of Fisher. She claimed that tea tastes better when your poor the milk into the cup first and the tea second. Fisher was baffled by this suggestion. He suggested an experiment -- Bristol would blind taste test a number of cups of tea and guess which ones were prepared milk-first and which ones were prepared tea-first. Fisher's hypothesis is, naturally, that she doesn't have this ability. We'll call it $H_0$.

Let's work through a number of scenarios to see how we might test $H_0$. Suppose Bristol manages to identify 3 out of 4 cups correctly. This evidence is ambiguous. She's done a little bit better than guessing at random but not quite as good as getting every cup right. If $H_0$ is correct, there is some chance that she manages to get 3 cups right. So we need a probability distribution to measure how likely this is, given $H_0$. Or, in probability notation, 

$$P(three \, cups \, correct | H_0)$$

This expression says "the probability of getting three cups (out of four) correct, assuming she has no special ability. The bar in the middle means "given" or "assuming". The first half of the expression is the outcome we are trying to ascertain the probability of (three cups). The second of the expression specifies our assumptions or our *model* of the situation. Finally the `P()` function takes all that information in and returns a probability.

At this point, you should try the problem yourself. What's the probability that she gets 3 cups correct if we assume she has no special tea discernment ability?

***
***

If Bristol has no special tea discernment ability, there is a 0.5 probability that she guesses a given up cup correctly. She's effectively flipping a coin to decide. Let's look at one wrong answer before we look at the right answer.

## One way to go wrong

We could reason like this: on the first cup, she has a 0.5 probability of being correct. On the second cup, she again has a 0.5 probability of being correct. On the third cup, she has the same. At this point, she can get the last cup wrong. So the probability of getting three cups right is just:

$$ 0.5 * 0.5 * 0.5 = 0.125 $$

The problem with this answer is that there is more than one way to get 3 cups right. You could do the first three cups, but you could could also do the last three cups. Or you could get one right, one wrong, and then two more right. So we need to consider a bigger set of possibilities.

Lots and lots of students give this answer on their first try. It's very natural. The point of here is just that probability is hard. It doesn't come intuitively. We can help ourselves develop better intuitions by writing simulations, something we'll turn to later today.

## Enumeration

Let's count all the ways she could end up with 3 right answers. Let C stand for correct and W stand for wrong. We'll just enumerate all the possibilities.


|4/4|3/4|2/4|1/4|0/4|
|---|---|---|---|---|
|CCCC | CCCW | CCWW | WWWC | WWWW |
| | CCWC | CWCW | WWCW | |
| | CWCC | CWWC | WCWW | |
| | WCCC | WCCW | CWWW | |
| | | WWCC | | |
| | | WCWC | | |

Now we just count up the number of possibilities that match our scenario and divide by the total number of possibilities.

$$ \frac{4}{16} = \frac{1}{4} = 0.25 $$

So it's a bit bigger than our original answer. It's not *that* unlikely she'll get 3 of 4 right. By contrast, it's more unlikely she gets them all right by good luck alone $\frac{1}{16} = 0.0625 = 6.25\% $. The important thing is that we've found $P(three \, cups \, correct | H_0) = \frac{1}{4}$.

## Visualizing the possibilities

We can visualize the probability distribution by turning each possible scenario into a frequency.

In [ ]:
# a list of all the possible outcomes - she could get 4 correct or 3 or 2 or ...

outcomes = [4,3,2,1,0] 

# the number of ways to realize those outcomes

counts = [1,4,6,4,1] 

# divide them all by 16 to get probabilities

frequency = [i / 16 for i in counts] 

# our old friend matplotlib

import matplotlib.pyplot as plt 

# a new type of plot, the bar graph

plt.bar(outcomes, frequency)

The plot shows us what the likelihood of all possible outcomes are, under the hypothesis that she has no special ability for discernment. Notice that 2 is the most likely outcome - this makes sense. If you have 50/50 chance of getting a cup right, you should get about half the cups right.

# Building a model

This approach, enumerating all possible events and counting up the events we care about, works. But it's tedious. Suppose we made the problem bigger. Suppose Bristol had to taste test eight tea cups and she got 6 correct. I'm not about to write down all 64 possible outcomes. It also has trouble once we modify the probability of success on each trial. Right now, the probability of success is 0.5. So each outcome is weighted equally. But once we work on more complex problems, we need to weight some columns more than others. 

We need solutions that scale. One strategy that always works is to write a simulation: if you can specify a story about what Lady Bristol is doing and convert it into code, you can simulate that code a large number of times to extract probabilities. It's a skill we'll use over and over again in this course. 

## A conceptual model

Before we can code, we need to get just a little bit clearer on what we think Lady Bristol is doing at each point in the experiment. Let's assume she's taste testing eight cups for most of the lecture.

On each cup, she has some probability of getting it correct. We'll call this probability *accuracy* or $a$. The probability is getting it wrong is the compliment of $a$. A compliment is another way of saying $1 - a$. Why $1 - a$? Let's think about some examples. Suppose she's really good at identifying cups, she gets them right 90% of the time. That implies she gets them wrong 10% of the time. And well, $1 - 0.9 = 0.1$. Alternatively, she might be guessing at random, or her accuracy is 50%. So the odds she gets it wrong are also 50%. Well, $1 - 0.5 = 0.5$.

We're also going to assume that her accuracy remains the same through the taste test. This is like saying she doesn't get better and she doesn't get worse as time goes on. If you think this assumption is a little funny, you are right - we'll revisit it next class and build a model that doesn't have to make assumptions like this.

## Simulating a single cup

With these assumptions on the table, let's build up our simulation, starting with a simulation of tasting a single cup. Let's say 0 represents getting it wrong and 1 respresents getting it right. So we need a function that returns 1 with probability equal to Bristol's accuracy or $a$. For now let's assume Bristol's accuracy is 0.5 or 50%.

In [ ]:
# Our old friend, import numpy as np

import numpy as np 

a = 0.5

np.random.choice([1,0],p=[a,1-a])

Try running this cell repeatedly. This will show you the range of outputs and give you an impression of how frequently they appear. Alternatively, we can assume Bristol is more accurate. Run the next cell repeatedly to verify that her accuracy has improved.

In [ ]:
a = 0.7

np.random.choice([1,0],p=[a,1-a])

`np.random.choice` is a function that lets you randomly select items from a list according to probabilities given by another list. So the outcomes (1 and 0) are in the first list, representing success and failure. The probabilities of those outcomes are in the second list. 

Notice the little `p=` in the second argument. This is the first time we've seen what's called a keyworded argument in a python function. Keyword arguments are optional. You can run `np.random.choice` without it.

In [ ]:
np.random.choice([1,0])

In our case, these two functions are equivalent. By default, the function assumes that each option has equal probability. So if there are two options, each one shows up 50% of the time. But if we want to modify that probability, we can pass it the optional `p=` argument. To see what the possible arguments of a function are, google search for the numpy documentation.

    Docstring:
    choice(a, size=None, replace=True, p=None)

    Generates a random sample from a given 1-D array
    
It shows you something called the docstring - a set of helpful reminders for what the parts of a function are and what it does. So this function has 3 optional keyword arguments and one required argument.

## Simulating a batch of cups

Let's have Bristol taste test 8 cups in a row. We can use our `for i in range()` structure to do this. First remind yourself about range objects

In [ ]:
# remind yourself about range objects

range(8)

In [ ]:
list(range(8))

Next we'll wrap our previous code in a `for` loop.

In [ ]:
# run this cell repeatedly until you get a sense for what the underlying mechanics are

a = 0.7

for i in range(8):
    
    outcome = np.random.choice([1,0],p=[a,1-a])
    print(outcome)

Although, we use `i` as the item in our loop, we do not actually use it in the calculations. That's okay. When we just want to repeat something 8 or 100 times, we use structures like this.

Let's pause for a moment to appreciate what we've accomplished. We have a very simple simulation that represents how Lady Bristol might drink cups of tea. This is our core goal for today. We'll spend the rest of the day dressing up our model and seeing what it can do for us. 

# Safety first

This model is very simple. But our models will quickly grow to be quite complex. So we need strategies for build up complex models in a way where we can understand what the code is doing. We are going to practice these strategies when the models are simple that way we know how to use them when the models are complex. We'll introduce two strategies today: one for managing your working memory and one for debugging. Let's look at doc strings and circle back around to debugging later in the lecture.

## Doc strings

One thing everyone struggles with is remembering what all the bits of our code do. Fortunately, we can write reminders to ourself, called doc strings. We already took at look at the doc string for `np.random.choice()`. But we can write our own doc strings for our own functions.

First, let's make some adjustment so our model is suitable for a function. Let's say there are two things we'll want to modify when we run our model, her accuracy $a$ and the number of cups she drinks $n$. Then, instead of printing, let's just have the function return a list of which cups she got right and which she got wrong.

In [ ]:
def simulate_bristol(a,n):

    experiment = []
    
    # notice a is missing. Instead of defining
    # at the start of the cell, it gets defined
    # when we specify the arguments of our function

    for i in range(n): # notice i replaced 8 with n
        
        outcome = np.random.choice([1,0],p=[a,1-a])
        experiment.append(outcome)
        
    return experiment

simulate_bristol(0.5,8)

Second, we can add a docstring by putting wrapping some text in three quotation marks like `''' text '''`. A good docstring tells you what the function outputs (or returns), what the arguments of the function do and any other handy information. Below is an example

In this class, I don't have strict rules about how to write docstrings. I know in CPSC 110 and other courses, they have a protocol for how to write docstrings and grade assignments to encourage. I won't be doing that here. So write what you find helpful.

In [ ]:
def simulate_bristol(a,n):
    '''Returns a list of 0s and 1s of lengeth n
    with 0 indicating failure and 1 indicating success.
    
    a = accuracy on each cup
    n = the number of cups'''

    experiment = []

    for i in range(n):
        
        outcome = np.random.choice([1,0],p=[a,1-a])
        experiment.append(outcome)
        
    return experiment

simulate_bristol(0.5,8)

Now that we have a function defined, it is easy to study how the behavior of the model changes across parameter values. When I'm working with a new model, I like to perform simple checks to make sure that the model exhibits expected behaviors. For example, if lady Bristol had 100% accuracy, she should get every cup correct. So we should get a list of eight ones. Confirm this is true:

Similarly, if accuracy = 0, we should get eight 0s.

Finally, let's check the `n` argument, which controls the number of cups she tastes. If we push `n` to 10, do we get a list of length 10?

# From simulations to probabilities

So far we have the ability to simulate a single trial of the tea experiment. But what we really want is a sense of how likely it is that Lady Bristol gets 6 cups right out of 8. This was our $P(6|H_{0})$ quantity we started off with. One way to find this number is to simulate our model a large number of times and count up how often Bristol gets 6 correct. 

If we only care about how often Bristol gets 6 correct, then the order of 1s and 0s doesn't really matter. Just the total number of 1s. So we can summarize a simulation by using `sum()`.

In [ ]:
sum(simulate_bristol(0.5,8))

Next, we need to be able to run many simulated experiments and record how often she gets various numbers of cups correct. Let's run 10 simulations and put the results in a list.

In [ ]:
all_trials = []

for i in range(10):
    
    correct = sum(simulate_bristol(0.5,8))
    all_trials.append(correct)
    
all_trials

Just like we wrapped our simulation in a function, we should wrap this in a function too. It will have three arguments. We'll need accuracy and the number of cups per simulation, just as before. But we also need be able to control how many simulated experiments we run too. We'll call this number precision. Once we get to estimating probabilities, we'll be to control how precise our estimates are by adjusting this number.

In [ ]:
def run_experiments(a,n,precision):
    '''Returns a list containing the number of cups lady bristol
    guessed correctly over many experiments
    
    a = accuracy on each cup
    n = number of cups per experiment
    precision = the number of experiments'''
    
    
    all_trials = []

    for i in range(precision): # add precision here

        correct = sum(simulate_bristol(a,n)) # add a and n here
        all_trials.append(correct)

    return all_trials

run_experiments(0.5,8,10)

The core idea behind simulation-based probability estimates works like this: the probability of an event is just the frequency of that event divided by the number of total events.

$$\frac{event}{total \, number \, of \, events}$$

So if we run our simulation 1000 times, we just need to count how many times Bristol got 6 correct and then divide it by 1000. So the final step is counting how many trials have exactly 6 correct guesses and dividing.

In [ ]:
all_trials = run_experiments(0.5,8,1000)
matches = all_trials.count(6)
matches

In [ ]:
# the final step is to convert this count into a frequency

matches / 1000

This is the desired probability. The chance of getting 6 out of 8 right is around 0.10-0.14 or $P(6|H_{0}) \approx 0.125$. Or, to return to a more real-world context, if we suppose Lady Bristol has no special ability to discern tea made milk-first and tea-first, there is about a $1/10$ chance she gets 6 cups correct. Not super likely, but not totally implausible either. This simulation approach might look fancy but it relies on the same intuition that we used to get our first answer with counting up all the possible combinations. Let's finish it off by streamlining the code. We'll move all the arguments up to the top so we can see them easily and move all the computations down to the bottom.

In [ ]:
a = 0.5
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)
matches = all_trials.count(k)
matches / precision

# Evaluating our simulation

In this course, we'll have the opportunity to build dozens of different models and extract probabilities from them. Each time we do, we need to do a variety of checks to make sure the model is working as intended. This helps us in a number of places: is the model realistic? If the predictions it makes are totally bizarre, that suggests maybe some of our assumptions were back. Is the model programmed correctly? Again, bizarre outputs can indicate that we've made a mistake somewhere. Finally, the assumptions might be okay and the program might be well-written but we do not understand why the model outputs what it does. The solution to all three of these programs is to just explore what the model does. So I'll guide you through a number of explorations but hopefully, as the semester proceeds, these start to be routine.

## Make sure the model matches things you already know

At the beginning of the lecture, we considered a case where Lady Bristol for 3 cups correct out of 4 and learned the probability of that, assuming she has no special skill, is 0.25. Does our model know this? Adjust the parameters to model an experiment with 4 cups and 3 correct guesses.

### Hide cell

In [ ]:
a = 0.5
n = 4
k = 3
precision = 1000

all_trials = run_experiments(a,n,precision)
matches = all_trials.count(k)
matches / precision

## Play with precision

You may notice that your answer isn't exactly 0.25 but it is close. This means either there is a mistake somewhere in the code or the precision isn't high enough. Try increasing the precision to 10,000 and re-asses.

In [ ]:
a = 0.5
n = 4
k = 3
precision = 10000

all_trials = run_experiments(a,n,precision)
matches = all_trials.count(k)
matches / precision

Again, try precision at 100000

In [ ]:
a = 0.5
n = 4
k = 3
precision = 100000

all_trials = run_experiments(a,n,precision)
matches = all_trials.count(k)
matches / precision

100000 is closer but not by much. You might also notice that it takes your computer much longer to return answer. Running 100000 simulated tea tasting experiments is computationally demanding. In general, we face a tradeoff: we can get more precise answers but we have to spend more time getting it. In this class, we'll always try to strike a balance. Rough estimates are okay if the alternative takes more than a few minutes to compute. But increasing precision from 100 to 1000 barely increases the compute time but massively increases the precision of the estimate.

## Play with accuracy

So far we've only been considering the scenario where Lady Bristol has no special ability. But what if she has a little bit of discernment? How much do the probabilities change if we she has a lot? Try increasing the accuracy and observe what happens to the number.

### hide cell

In [ ]:
a = 0.75
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)
matches = all_trials.count(k)
matches / precision

## Plots

Plots can be tremendously helpful in understanding and interpreting our model. I want to produce a bar graph to summarize our findings. It would be nice if we could see the probability of any of the outcomes, not just the one we are immediately interested in. That graph would be a depiction of a *binomial probability distribution*. 

To pull this off, we need to count how many trials had 1 success, how many had 2 successes, 3 successes and so on. Actually zero through 8. So we'll need to run a loop the counter and we'll need to make sure that loop is as long as the number of possible outcomes. Let me show you:

In [ ]:
a = 0.75
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)

probs = []

for i in range(n+1): # we'll use a range starting at zero and continuing through n

    matches = all_trials.count(i)
    prob = matches / precision
    probs.append(prob)
    
probs

after that, we can just plot the list and use the same range. The bar plot shows us the probability of all possible outcomes, assuming our specific parameter settings

In [ ]:
plt.bar(range(n+1),probs)
plt.show()

Take a minute to play with this distribution chart. Adjust parameters and see how it behaves. This is a good opportunity to explore on your own.

In [ ]:
a = 0.5
n = 8
k = 6
precision = 10000

all_trials = run_experiments(a,n,precision)

probs = []

for i in range(n+1): # we'll use a range starting at zero and continuing through n+1

    matches = all_trials.count(i)
    prob = matches / precision
    probs.append(prob)
    
plt.bar(range(n+1),probs)
plt.show()

# Debugging with print()

We've written a lot of code today! In particular, our code is tightly interwoven. Each piece gets re-used by later pieces. This makes our project a bit fragile - if one piece is malfunctioning, it will hurt the other pieces. It might hard to identify where the problem is because, although our programming language throws an error on some line, the real mistake is further upstream. Let me give you an example.

In [ ]:
a = 0.5
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)

probs = []

for i in range(n+1): # we'll use a range starting at zero and continuing through n+1

    matches = all_trials.count(i)
    prob = matches / precision
    probs.append(prob)
    
plt.bar(range(n),probs)
plt.show()

What does this one even mean? Cannot be broadcast to a single shape? I'm going to walk you through how to understand error reports. At the top of this error, you'll see part of the code you've written. This is our code.

         14     probs.append(prob)
         15 
    ---> 16 plt.bar(range(n),probs)
         17 plt.show()
         
So the line indicates that plt.bar is having trouble. Then the rest of the error report is showing the code that lives inside `plt.bar()`. We didn't write this stuff. A programmer with the matplotlib team wrote it for us. 

       2365         x, height, width=0.8, bottom=None, *, align='center',
       2366         data=None, **kwargs):
    -> 2367     return gca().bar(
       2368         x, height, width=width, bottom=bottom, align=align,
       2369         **({"data": data} if data is not None else {}), **kwargs)
       
In general, you do not need to try to read this code, the code inside our packages. It's been extremely well tested and it's very unlikely the bug is inside `plt.bar()`. 

Instead, the first thing I want to do is add `print()` statements to the code so I can see what the arguments look like by the time they make it do `plt.bar()`.

In [ ]:
a = 0.5
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)

probs = []

for i in range(n+1): # we'll use a range starting at zero and continuing through n+1

    matches = all_trials.count(i)
    prob = matches / precision
    probs.append(prob)
    
print(range(n))
print(probs)
    
#plt.bar(range(n),probs) commented out to the surpress the error messages.
#plt.show();

shape errors usually refer to how many rows and columns are in lists. Mismatch means one list has a different size than the other. If I have 8 x coordinates but 9 y coordinates, the plotting package does not know where to put that 9th y coordinate. So let's check the lengths.

In [ ]:
a = 0.5
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)

probs = []

for i in range(n+1): # we'll use a range starting at zero and continuing through n+1

    matches = all_trials.count(i)
    prob = matches / precision
    probs.append(prob)
    
print(len(range(n)))
print(len(probs))
    
#plt.bar(range(n),probs) commented out to the surpress the error messages.
#plt.show();

Yup, 8 and 9. So we need to make x longer somehow. But in which direction? We know that the possible outcomes for Lady bristol are 0, 1, 2, 3, 4, 5, 6, 7, 8. So let's unpack the range to see where it goes.

In [ ]:
a = 0.5
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)

probs = []

for i in range(n+1): # we'll use a range starting at zero and continuing through n+1

    matches = all_trials.count(i)
    prob = matches / precision
    probs.append(prob)
    
print(list(range(n)))
print(probs)
    
#plt.bar(range(n),probs) commented out to the surpress the error messages.
#plt.show();

Ahh so it's missing the last possible outcome. Notice that, in the `for i in range()` structure, we used n+1. That's a trick to make sure the range continues for long enough. Sure enough, if we add the n+1 back in, our code functions again.

In [ ]:
a = 0.5
n = 8
k = 6
precision = 1000

all_trials = run_experiments(a,n,precision)

probs = []

for i in range(n+1): # we'll use a range starting at zero and continuing through n+1

    matches = all_trials.count(i)
    prob = matches / precision
    probs.append(prob)
    
plt.bar(range(n+1),probs)
plt.show();

That's all for today. Next class we'll connect these skills with the problem of induction and learn about what statistical significance really means.